
# 🧪 05 — RAG Hands-On (Node.js, Option A)

This notebook is the **Node.js twin** of the Python RAG lab.

Goal:

- Show you how to build a **minimal but real** RAG pipeline in Node.js:
  - load local docs (same ones as Python lab)
  - chunk text
  - embed with OpenAI (and optionally local embeddings)
  - store in a vector DB (Chroma)
  - retrieve and answer questions with citations
  - prepare for Agentic RAG / MCP integration later

> You’ll copy the code snippets from this notebook into your Node project files.



## 0. Node Project Setup

In your terminal:

```bash
mkdir rag-node-lab
cd rag-node-lab

npm init -y

# Core deps
npm install openai @langchain/core @langchain/community chromadb dotenv

# (Optional) local embeddings, if you want offline models later
# npm install @xenova/transformers
```

Create a `.env` file:

```bash
OPENAI_API_KEY=sk-...
```

And add this to `package.json` to enable ES modules if needed:

```jsonc
{
  "type": "module",
  "scripts": {
    "start": "node src/index.js"
  }
}
```



## 1. Sample Documents (Shared with Python Lab)

We will reuse the same sample documents from the Python lab:

- `data/sample_docs/finance_intro.md`
- `data/sample_docs/health_intro.md`
- `data/sample_docs/legal_clause.md`
- `data/sample_docs/code_sample.py`

You can point your Node scripts at the same folder if your repo layout is:

```text
rag_universe/
  data/
    sample_docs/
      finance_intro.md
      health_intro.md
      legal_clause.md
      code_sample.py
  notebooks/
    04_RAG_HandsOn_Python.ipynb
    05_RAG_HandsOn_Node.ipynb
```



## 2. Project Structure (Node)

Suggested structure for this lab:

```text
rag-node-lab/
  src/
    config.js
    loaders.js
    chunker.js
    embeddings.js
    vectorStore.js
    rag.js
    index.js
  .env
  package.json
```

You can merge this structure into your bigger **RAG Universe** repo later.



## 3. `src/config.js` — Configuration Helper

```js
// src/config.js
import 'dotenv/config';

export const OPENAI_API_KEY = process.env.OPENAI_API_KEY;

if (!OPENAI_API_KEY) {
  console.warn('⚠️ OPENAI_API_KEY is not set. Some features will not work.');
}
```



## 4. `src/loaders.js` — Load Local Files

We’ll use Node's `fs` and `path` to read `.md` and `.py` files.

```js
// src/loaders.js
import fs from 'fs';
import path from 'path';
import { fileURLToPath } from 'url';

const __filename = fileURLToPath(import.meta.url);
const __dirname = path.dirname(__filename);

export function loadSampleDocs(relativeDir = '../data/sample_docs') {
  const folder = path.resolve(__dirname, relativeDir);
  const entries = fs.readdirSync(folder, { withFileTypes: true });

  const docs = [];

  for (const entry of entries) {
    if (!entry.isFile()) continue;
    const ext = path.extname(entry.name);
    if (!['.md', '.py', '.txt'].includes(ext)) continue;

    const fullPath = path.join(folder, entry.name);
    const content = fs.readFileSync(fullPath, 'utf-8');
    docs.push({
      content,
      metadata: { source: fullPath },
    });
  }

  return docs;
}
```

You can test this via a small script:

```js
// src/debugLoad.js
import { loadSampleDocs } from './loaders.js';

console.log(loadSampleDocs());
```



## 5. `src/chunker.js` — Simple Chunking

We’ll implement a simple character-based chunker.

```js
// src/chunker.js

export function chunkDocuments(docs, chunkSize = 500, overlap = 100) {
  const chunks = [];

  for (const doc of docs) {
    const text = doc.content;
    let start = 0;

    while (start < text.length) {
      const end = Math.min(start + chunkSize, text.length);
      const chunkText = text.slice(start, end);
      chunks.push({
        content: chunkText,
        metadata: { ...doc.metadata },
      });
      start = end - overlap;
      if (start < 0) start = 0;
      if (start >= text.length) break;
    }
  }

  return chunks;
}
```



## 6. `src/embeddings.js` — Embedding Wrapper

We’ll start with OpenAI embeddings via `openai` and later you can plug in local models.

```js
// src/embeddings.js
import OpenAI from 'openai';
import { OPENAI_API_KEY } from './config.js';

export class Embedder {
  constructor(model = 'openai') {
    this.modelType = model;
    this.client = null;
  }

  ensureClient() {
    if (!this.client) {
      if (!OPENAI_API_KEY) {
        throw new Error('OPENAI_API_KEY is not set');
      }
      this.client = new OpenAI({ apiKey: OPENAI_API_KEY });
    }
  }

  async embedTexts(texts) {
    if (this.modelType === 'openai') {
      this.ensureClient();
      const response = await this.client.embeddings.create({
        model: 'text-embedding-3-large',
        input: texts,
      });
      return response.data.map(d => d.embedding);
    }

    // Placeholder for local embeddings:
    // if (this.modelType === 'local') { ... }

    throw new Error(`Unknown embedding model type: ${this.modelType}`);
  }
}
```



## 7. `src/vectorStore.js` — Chroma Vector Store

We’ll use the JS bindings for Chroma.

```js
// src/vectorStore.js
import { ChromaClient } from 'chromadb';

export class ChromaVectorStore {
  constructor(collectionName = 'rag_demo') {
    this.client = new ChromaClient();
    this.collectionName = collectionName;
    this.collection = null;
  }

  async init() {
    this.collection = await this.client.getOrCreateCollection({
      name: this.collectionName,
    });
  }

  async addDocuments(chunks, embeddings) {
    if (!this.collection) {
      await this.init();
    }
    const ids = chunks.map((_, i) => `chunk-${i}`);
    const texts = chunks.map(c => c.content);
    const metadatas = chunks.map(c => c.metadata);

    await this.collection.add({
      ids,
      documents: texts,
      metadatas,
      embeddings,
    });
  }

  async similaritySearch(queryEmbedding, k = 5) {
    if (!this.collection) {
      await this.init();
    }
    const result = await this.collection.query({
      queryEmbeddings: [queryEmbedding],
      nResults: k,
    });

    const docs = [];
    const { documents, metadatas } = result;

    if (documents && documents[0]) {
      for (let i = 0; i < documents[0].length; i++) {
        docs.push({
          pageContent: documents[0][i],
          metadata: metadatas ? metadatas[0][i] : {},
        });
      }
    }

    return docs;
  }
}
```



## 8. `src/rag.js` — Retrieval + Answer

Now we glue everything together:

```js
// src/rag.js
import OpenAI from 'openai';
import { loadSampleDocs } from './loaders.js';
import { chunkDocuments } from './chunker.js';
import { Embedder } from './embeddings.js';
import { ChromaVectorStore } from './vectorStore.js';
import { OPENAI_API_KEY } from './config.js';

const openaiClient = new OpenAI({ apiKey: OPENAI_API_KEY });

export async function buildRagIndex() {
  const docs = loadSampleDocs(); // local sample docs
  const chunks = chunkDocuments(docs, 400, 80);

  const embedder = new Embedder('openai');
  const texts = chunks.map(c => c.content);
  const embeddings = await embedder.embedTexts(texts);

  const store = new ChromaVectorStore('rag_demo_node');
  await store.addDocuments(chunks, embeddings);

  return store;
}

function buildContext(docs, maxChars = 2000) {
  let total = 0;
  const parts = [];

  for (const d of docs) {
    const snippet = d.pageContent.slice(0, 500);
    const src = d.metadata?.source ?? 'unknown';
    const block = `Source: ${src}\n${snippet}`;
    if (total + block.length > maxChars) break;
    parts.push(block);
    total += block.length;
  }

  return parts.join('\n\n---\n\n');
}

export async function ragAnswer(store, query, k = 5, model = 'gpt-4o-mini') {
  // 1) Embed query
  const embedder = new Embedder('openai');
  const [queryEmbedding] = await embedder.embedTexts([query]);

  // 2) Retrieve
  const docs = await store.similaritySearch(queryEmbedding, k);
  const context = buildContext(docs);

  // 3) Generate
  const systemPrompt = [
    'You are a careful RAG assistant.',
    'Use ONLY the provided context to answer.',
    'If the context is insufficient, say you are not sure.',
    'Always include brief citations mentioning the source paths.',
  ].join(' ');

  const completion = await openaiClient.chat.completions.create({
    model,
    messages: [
      { role: 'system', content: systemPrompt },
      {
        role: 'user',
        content: `Question: ${query}\n\nContext:\n${context}`,
      },
    ],
  });

  return completion.choices[0]?.message?.content ?? '';
}
```



## 9. `src/index.js` — Run a Demo Query

```js
// src/index.js
import { buildRagIndex, ragAnswer } from './rag.js';

async function main() {
  const store = await buildRagIndex();
  const query = 'Explain dollar-cost averaging in simple terms.';
  const answer = await ragAnswer(store, query, 5);
  console.log('Q:', query);
  console.log('\nA:', answer);
}

main().catch(err => {
  console.error(err);
  process.exit(1);
});
```

Run:

```bash
node src/index.js
```



## 10. Next Steps (Node.js)

From here, you can:

- Swap Chroma for **Pinecone** using the JS client
- Add a **reranker** (e.g., Cohere or LLM-based)
- Implement a **conversational wrapper** that keeps short-term history
- Integrate with your **Agents Universe** (Agentic RAG in Node)
- Expose as an API using **Express**, **Fastify**, or **Hono**
- Build a simple **React / Next.js** UI that calls this RAG backend

This notebook plus the Python one give you:

- A **cross-language mental model**
- A clear, runnable starting point for Node-based RAG pipelines
- A direct bridge into your **RAG + Agents + MCP** architecture.
